In [5]:
import pandas as pd

df1 = pd.read_csv('/Users/ylpan/Documents/GitHub/GxE-Interaction-Regression-Analysis/data/AMS578_Y_1415.csv')
df2 = pd.read_csv('/Users/ylpan/Documents/GitHub/GxE-Interaction-Regression-Analysis/data/AMS578_E_1415.csv')
df3 = pd.read_csv('/Users/ylpan/Documents/GitHub/GxE-Interaction-Regression-Analysis/data/AMS578_G_1415.csv')

In [7]:
combined_df = pd.concat([df1, df2, df3], axis=1)
# Display the combined dataset
print(combined_df.head())

           Y         E1         E2         E3         E4         E5  \
0  22.886871  50.563010  49.887368  50.673395  50.448073  49.666366   
1        NaN  49.707738  50.183333  48.324652  51.161940  50.786052   
2  22.731857  49.576813  49.456161  49.685875  51.128247  49.616270   
3  23.081865  48.478951  50.091171  49.493196  49.680849  48.339918   
4        NaN  50.625206  50.533741  49.522284  49.690645  49.971374   

          E6   G1   G2   G3  ...  G11  G12  G13  G14  G15  G16  G17  G18  G19  \
0  49.843056  1.0  1.0  1.0  ...  1.0  1.0  1.0  1.0  1.0  0.0  1.0  0.0  NaN   
1  48.940399  1.0  1.0  1.0  ...  1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0  0.0   
2  51.081135  1.0  1.0  0.0  ...  0.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0   
3  50.783511  1.0  1.0  0.0  ...  1.0  1.0  0.0  1.0  1.0  1.0  1.0  0.0  1.0   
4  50.179141  1.0  NaN  1.0  ...  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   

   G20  
0  1.0  
1  1.0  
2  1.0  
3  1.0  
4  1.0  

[5 rows x 27 columns]


In [10]:

# %pip install scikit-learn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Initialize the IterativeImputer
imputer = IterativeImputer(max_iter=10, random_state=0)

# Perform the imputation
imputed_df = pd.DataFrame(imputer.fit_transform(combined_df), columns=combined_df.columns)

# Display the imputed dataset
print(imputed_df.head())

           Y         E1         E2         E3         E4         E5  \
0  22.886871  50.563010  49.887368  50.673395  50.448073  49.666366   
1  23.034115  49.707738  50.183333  48.324652  51.161940  50.786052   
2  22.731857  49.576813  49.456161  49.685875  51.128247  49.616270   
3  23.081865  48.478951  50.091171  49.493196  49.680849  48.339918   
4  23.269285  50.625206  50.533741  49.522284  49.690645  49.971374   

          E6   G1        G2   G3  ...  G11  G12  G13  G14  G15  G16  G17  G18  \
0  49.843056  1.0  1.000000  1.0  ...  1.0  1.0  1.0  1.0  1.0  0.0  1.0  0.0   
1  48.940399  1.0  1.000000  1.0  ...  1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0   
2  51.081135  1.0  1.000000  0.0  ...  0.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0   
3  50.783511  1.0  1.000000  0.0  ...  1.0  1.0  0.0  1.0  1.0  1.0  1.0  0.0   
4  50.179141  1.0  0.809175  1.0  ...  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   

        G19  G20  
0  0.776447  1.0  
1  0.000000  1.0  
2  1.000000  1.0  
3  1.00000

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# Define the independent variables (E and G columns)
X = imputed_df.drop(columns=['Y'])
y = imputed_df['Y']

# Initialize the linear regression model
model = LinearRegression()

# Initialize RFE with the linear regression model and specify the number of features to select
rfe = RFE(model, n_features_to_select=10)  # You can adjust the number of features to select

# Fit RFE
rfe = rfe.fit(X, y)

# Get the selected features
selected_features = X.columns[rfe.support_]

print("Selected important features:")
print(selected_features)

Selected important features:
Index(['E1', 'G1', 'G5', 'G6', 'G9', 'G10', 'G11', 'G15', 'G16', 'G17'], dtype='object')


In [15]:
from sklearn.preprocessing import PolynomialFeatures

# Initialize PolynomialFeatures with degree 2 to include interaction terms
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

# Fit and transform the data to include interaction terms
X_interaction = poly.fit_transform(X[selected_features])

# Create a DataFrame with the interaction terms
interaction_feature_names = poly.get_feature_names_out(selected_features)
X_interaction_df = pd.DataFrame(X_interaction, columns=interaction_feature_names)

# Display the DataFrame with interaction terms
# print(X_interaction_df.head())

In [16]:
# Initialize RFE with the linear regression model and specify the number of features to select
rfe_interaction = RFE(model, n_features_to_select=10)  # Adjust the number of features to select as needed

# Fit RFE on the interaction terms
rfe_interaction = rfe_interaction.fit(X_interaction_df, y)

# Get the selected features
selected_interaction_features = X_interaction_df.columns[rfe_interaction.support_]

print("Selected important features with interaction terms:")
print(selected_interaction_features)


Selected important features with interaction terms:
Index(['E1', 'G1', 'G11', 'G16', 'G17', 'G1 G16', 'G5 G9', 'G5 G16', 'G10 G17',
       'G11 G17'],
      dtype='object')


In [19]:
# Define the selected features for the regression model
selected_features_for_model = list(selected_interaction_features)

# Extract the data for the selected features
X_selected = X_interaction_df[selected_features_for_model]

# Initialize the linear regression model
model_selected = LinearRegression()

# Fit the model
model_selected.fit(X_selected, y)

# Display the coefficients of the model
print("Coefficients of the regression model:")
print(model_selected.coef_)

# Display the intercept of the model
print("Intercept of the regression model:")
print(model_selected.intercept_)

Coefficients of the regression model:
[ 0.1343256   0.25317599 -0.10383825  0.36846844 -0.1018192   0.10546498
  0.04913649 -0.07717179  0.04082774  0.10949093]
Intercept of the regression model:
15.829445664263684


In [20]:
from sklearn.metrics import r2_score

# Predict the target variable using the selected model
y_pred = model_selected.predict(X_selected)

# Calculate the R^2 score
r2 = r2_score(y, y_pred)

print("R^2 score of the regression model:")
print(r2)

R^2 score of the regression model:
0.6290755347683574
